1. Bibliotecas e Leitura de Dados

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
import string
import joblib
import nltk
from sklearn.model_selection import train_test_split
import torch

1.1. Leitura do dataset de mensagens fraudulentas

In [170]:
# Lê o dataset com mensagens classificadas como fraudulentas
dataset_pd = pd.read_csv("..\\Datasets\\fraudes.csv", encoding='utf-8')
df_fraudes = pd.DataFrame(dataset_pd)
df_fraudes.head()

,titulo,assunto,descricao,url
0,FRAUDE - Perfil Profissional,Assunto da mensagem: Perfil Profissional Compl...,"\r\n\r\nOlá,\r\n\r\nEnvio minhas informações p...",https://catalogodefraudes.rnp.br/frauds/16581
1,FRAUDE - Phishing de Cartão de Crédito,Assunto da mensagem: Convite Especial: Bradesc...,\r\n\r\n\r\nConvite Exclusivo: Bradesco Americ...,https://catalogodefraudes.rnp.br/frauds/16575
2,FRAUDE - Phishing por Atualização de Email,Assunto da mensagem: Re: NF-e: <ID>,\r\n\r\n\r\nHá uma alteração e atualização em ...,https://catalogodefraudes.rnp.br/frauds/16576
3,FRAUDE - Nota Fiscal Eletrônica,Assunto da mensagem: Seu pedido foi realizado ...,\r\n\r\n\r\n\r\n<ID>\r\nNota Fiscal Disponível...,https://catalogodefraudes.rnp.br/frauds/16580
4,FRAUDE - Atualização de E-mail,Assunto da mensagem: Notificação de cobrança /...,\r\n\r\n\r\nHá uma alteração e atualização em ...,https://catalogodefraudes.rnp.br/frauds/16574


In [171]:
# Adiciona a coluna "Fraude" com valor 1 (indicando que é fraude)
df_fraudes['Fraude'] = 1
df_fraudes.head()

,titulo,assunto,descricao,url,Fraude
0,FRAUDE - Perfil Profissional,Assunto da mensagem: Perfil Profissional Compl...,"\r\n\r\nOlá,\r\n\r\nEnvio minhas informações p...",https://catalogodefraudes.rnp.br/frauds/16581,1
1,FRAUDE - Phishing de Cartão de Crédito,Assunto da mensagem: Convite Especial: Bradesc...,\r\n\r\n\r\nConvite Exclusivo: Bradesco Americ...,https://catalogodefraudes.rnp.br/frauds/16575,1
2,FRAUDE - Phishing por Atualização de Email,Assunto da mensagem: Re: NF-e: <ID>,\r\n\r\n\r\nHá uma alteração e atualização em ...,https://catalogodefraudes.rnp.br/frauds/16576,1
3,FRAUDE - Nota Fiscal Eletrônica,Assunto da mensagem: Seu pedido foi realizado ...,\r\n\r\n\r\n\r\n<ID>\r\nNota Fiscal Disponível...,https://catalogodefraudes.rnp.br/frauds/16580,1
4,FRAUDE - Atualização de E-mail,Assunto da mensagem: Notificação de cobrança /...,\r\n\r\n\r\nHá uma alteração e atualização em ...,https://catalogodefraudes.rnp.br/frauds/16574,1


In [172]:
# Remove colunas irrelevantes
df_fraudes.drop(['titulo', 'assunto', 'url'], axis=1, inplace=True)
df_fraudes.head()

,descricao,Fraude
0,"\r\n\r\nOlá,\r\n\r\nEnvio minhas informações p...",1
1,\r\n\r\n\r\nConvite Exclusivo: Bradesco Americ...,1
2,\r\n\r\n\r\nHá uma alteração e atualização em ...,1
3,\r\n\r\n\r\n\r\n<ID>\r\nNota Fiscal Disponível...,1
4,\r\n\r\n\r\nHá uma alteração e atualização em ...,1


In [173]:
# Remove duplicatas e valores nulos
df_fraudes.drop_duplicates(inplace=True)
df_fraudes = df_fraudes.dropna().reset_index(drop=True)

print(df_fraudes.shape)
df_fraudes.head()

(13893, 2)


,descricao,Fraude
0,"\r\n\r\nOlá,\r\n\r\nEnvio minhas informações p...",1
1,\r\n\r\n\r\nConvite Exclusivo: Bradesco Americ...,1
2,\r\n\r\n\r\nHá uma alteração e atualização em ...,1
3,\r\n\r\n\r\n\r\n<ID>\r\nNota Fiscal Disponível...,1
4,\r\n\r\n\r\nAtenção usuário da conta de webmai...,1


In [174]:
# Reduz o tamanho do dataset para evitar desbalanceamento ou sobrecarga
df_fraudes = df_fraudes[:3985]
df_fraudes.shape

(3985, 2)

1.2. Leitura do dataset de mensagens legítimas

In [175]:
#Leitura do arquivo csv de mensagens não fraudulentas
dataset_pd2 = pd.read_csv("..\\Datasets\\df_nao_fraude_tratada2_4354.csv", encoding='utf-8')
dataset_pd2.head()


,Unnamed: 0,descricao,Fraude
0,0,qui abr agradecemos por escolher o pop total r...,0
1,1,sex abr agradecemos por escolher o pop total r...,0
2,2,seg abr agradecemos por escolher o pop total r...,0
3,3,qui mai agradecemos por escolher o pop total r...,0
4,4,qua mai agradecemos por escolher o pop total r...,0


In [176]:
# Remove colunas irrelevantes
df_legitimo = dataset_pd2.drop("Unnamed: 0", axis = 1)
df_legitimo.head()

,descricao,Fraude
0,qui abr agradecemos por escolher o pop total r...,0
1,sex abr agradecemos por escolher o pop total r...,0
2,seg abr agradecemos por escolher o pop total r...,0
3,qui mai agradecemos por escolher o pop total r...,0
4,qua mai agradecemos por escolher o pop total r...,0


In [177]:
# Remove duplicatas e valores nulos
df_legitimo.drop_duplicates(inplace=True)
df_legitimo = df_legitimo.dropna().reset_index(drop=True)

print(df_legitimo.shape)
df_legitimo.head()

(3636, 2)


,descricao,Fraude
0,qui abr agradecemos por escolher o pop total r...,0
1,sex abr agradecemos por escolher o pop total r...,0
2,seg abr agradecemos por escolher o pop total r...,0
3,qui mai agradecemos por escolher o pop total r...,0
4,qua mai agradecemos por escolher o pop total r...,0


Tratamento do segundo dataset de mensagens legitimas simuladas

In [178]:
#Leitura do arquivo csv de mensagens não fraudulentas 2
dataset_pd3 = pd.read_csv("..\\Datasets\\novo_df_legitimo.csv", encoding='utf-8')
dataset_pd3.head()

,Unnamed: 0,descricao,Fraude
0,1,"Olá, informamos que sua fatura do cartão de cr...",0
1,2,"Seu pagamento de boleto no valor de R$ 250,00 ...",0
2,3,"Prezado cliente, seu extrato mensal já está di...",0
3,4,Compra aprovada! Você realizou uma compra de R...,0
4,5,"Olá, recebemos sua transferência de R$ 500,00....",0


In [179]:
# Remove colunas irrelevantes 2
df_legitimo2 = dataset_pd3.drop("Unnamed: 0", axis = 1)
df_legitimo2.head()

,descricao,Fraude
0,"Olá, informamos que sua fatura do cartão de cr...",0
1,"Seu pagamento de boleto no valor de R$ 250,00 ...",0
2,"Prezado cliente, seu extrato mensal já está di...",0
3,Compra aprovada! Você realizou uma compra de R...,0
4,"Olá, recebemos sua transferência de R$ 500,00....",0


In [180]:
# Remove duplicatas e valores nulos 2
df_legitimo2.drop_duplicates(inplace=True)
df_legitimo2 = df_legitimo2.dropna().reset_index(drop=True)

print(df_legitimo2.shape)
df_legitimo2.head()

(349, 2)


,descricao,Fraude
0,"Olá, informamos que sua fatura do cartão de cr...",0
1,"Seu pagamento de boleto no valor de R$ 250,00 ...",0
2,"Prezado cliente, seu extrato mensal já está di...",0
3,Compra aprovada! Você realizou uma compra de R...,0
4,"Olá, recebemos sua transferência de R$ 500,00....",0


2. Pré-processamento de texto com NLTK

In [ ]:
#baixando e carregando uma lista de stopwords (palavras irrelevantes) em português da biblioteca NLTK (Natural Language Toolkit)

nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nando\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [182]:
# Função para remover pontuações, acentos e transformar para minúsculas
def process_text_conteudo(conteudo):
  import unicodedata
  
  # Remove pontuação
  nopont = [char for char in conteudo if char not in string.punctuation]

  # Converte para minúsculas
  nopont = ''.join(nopont).lower()
  
  # Remove acentos
  nopont = unicodedata.normalize('NFKD', nopont).encode('ASCII', 'ignore').decode('ASCII')

  return nopont


In [183]:
# Função de tokenização (opcional para vetorizador)
def tokenize_text(conteudo):
    return conteudo.split()

In [184]:
# Aplica o pré-processamento às mensagens
df_fraudes['descricao'] = df_fraudes['descricao'].apply(process_text_conteudo)

df_legitimo['descricao'] = df_legitimo['descricao'].apply(process_text_conteudo)

df_legitimo2['descricao'] = df_legitimo2['descricao'].apply(process_text_conteudo)

print(df_fraudes.head())
print(df_legitimo.head())
print(df_legitimo2.head())

                                           descricao  Fraude
0  \r\n\r\nola\r\n\r\nenvio minhas informacoes pr...       1
1  \r\n\r\n\r\nconvite exclusivo bradesco america...       1
2  \r\n\r\n\r\nha uma alteracao e atualizacao em ...       1
3  \r\n\r\n\r\n\r\nid\r\nnota fiscal disponivel\r...       1
4  \r\n\r\n\r\natencao usuario da conta de webmai...       1
                                           descricao  Fraude
0  qui abr agradecemos por escolher o pop total r...       0
1  sex abr agradecemos por escolher o pop total r...       0
2  seg abr agradecemos por escolher o pop total r...       0
3  qui mai agradecemos por escolher o pop total r...       0
4  qua mai agradecemos por escolher o pop total r...       0
                                           descricao  Fraude
0  ola informamos que sua fatura do cartao de cre...       0
1  seu pagamento de boleto no valor de r 25000 fo...       0
2  prezado cliente seu extrato mensal ja esta dis...       0
3  compra aprovada voce 

3. Salvando os datasets processados

In [185]:
df_fraudes.to_csv('..\\Datasets\\df_fraudes_tratada.csv', index=True)
df_legitimo.to_csv('..\\Datasets\\df_legitimo_tratado.csv', index=True)
df_legitimo2.to_csv('..\\Datasets\\df_legitimo2_tratado.csv', index=True)

4. Unificação dos datasets

In [186]:
data_frame = pd.concat([df_fraudes, df_legitimo, df_legitimo2])

print(data_frame.shape)
data_frame.head

(7970, 2)


<bound method NDFrame.head of                                              descricao  Fraude
0    \r\n\r\nola\r\n\r\nenvio minhas informacoes pr...       1
1    \r\n\r\n\r\nconvite exclusivo bradesco america...       1
2    \r\n\r\n\r\nha uma alteracao e atualizacao em ...       1
3    \r\n\r\n\r\n\r\nid\r\nnota fiscal disponivel\r...       1
4    \r\n\r\n\r\natencao usuario da conta de webmai...       1
..                                                 ...     ...
344  pix recebido r 18000 de felipe andrade consult...       0
345    pagamento de boleto r 11500 para faculdade xpto       0
346  transferencia entre bancos r 90000 enviados pa...       0
347  pagamento de assinatura disney r 3390 debitado...       0
348  recarga de celular concluida r 4000 para 11 98...       0

[7970 rows x 2 columns]>

In [187]:
data_frame.to_csv('..\\Datasets\\data_frame.csv', index=True)

5. Treinamento do modelo de IA

In [188]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Transforma os textos em vetores TF-IDF (matriz esparsa)
vectorizer = TfidfVectorizer(
    max_df=0.85, 
    min_df=5, 
    ngram_range=(1,2), 
    sublinear_tf=True
)

data_frame['descricao'] = data_frame['descricao'].apply(process_text_conteudo)
vect = vectorizer.fit_transform(data_frame['descricao'].values.astype('U'))


In [189]:
print(vect.shape)

(7970, 33177)


6. Separação de dados para treino e teste

In [ ]:

x_train, x_test, y_train, y_test = train_test_split(vect, data_frame['Fraude'], test_size = 0.20, random_state = 0)

7. Treinamento com Naive Bayes

In [191]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB().fit(x_train, y_train)

8. Avaliação do modelo

In [192]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Avaliação com os dados de treino
print('Avaliação com os dados de treino')
pred = classifier.predict(x_train)
print(classification_report(y_train, pred))
print(confusion_matrix(y_train, pred))
print('Accuracy:', accuracy_score(y_train, pred))

print('\n\n')

# Avaliação com os dados de teste
print('Avaliação com os dados de teste')
pred = classifier.predict(x_test)
print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))
print('Accuracy:', accuracy_score(y_test, pred))

Avaliação com os dados de treino
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      3203
           1       0.97      0.98      0.98      3173

    accuracy                           0.97      6376
   macro avg       0.98      0.97      0.97      6376
weighted avg       0.98      0.97      0.97      6376

[[3094  109]
 [  51 3122]]
Accuracy: 0.9749058971141782



Avaliação com os dados de teste
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       782
           1       0.96      0.98      0.97       812

    accuracy                           0.97      1594
   macro avg       0.97      0.97      0.97      1594
weighted avg       0.97      0.97      0.97      1594

[[751  31]
 [ 17 795]]
Accuracy: 0.9698870765370138


9. Salvando modelo e vetorizar

In [193]:
joblib.dump(classifier, 'modelo.joblib')
joblib.dump(vectorizer, 'tfidf_vectorizer.joblib')

['tfidf_vectorizer.joblib']

10. Teste com uma mensagem manual

In [194]:
# Exemplo de mensagem (legítima ou fraude)

# O primeio exemplo é um e-mail normal
dados1 = {"descricao":"Olá, pessoal, pronto para um novo desafio DEV? Hoje sai o terceiro vídeo da semana devepoler. Lembrando que esta é uma aula para todos os níveis de conhecimento. Ou seja, se você está no nível básico, intermediário ou avançado, a aula será bem produtiva e você aprenderá muito sobre vários temas. Como Nodejs, MongoDB, VueJS e muito mais: esta aula ficará no ar somente durante 7 dias, então recomendo assistir agora mesmo, pois se você deixar para depois, ela poderá não estar mais disponível!  Equipe xyz"}

dados2 = {"descricao" : "Esse email é legitimo."}

dados5 = {"descricao" : "Oi, Luiz Além de antecipar esse dinheiro sem nenhuma burocracia e com taxa de apenas 1,24% ao mês, o dinheiro cai na sua conta muito rápido. Dê mais pique para sua vida. Antecipar agora. Para solicitar a antecipação do FGTS é necessário optar pela modalidade de Saque Aniversário do FGTS e a concessão está sujeita à disponibilidade de saldo na sua conta FGTS e aprovação de crédito. As taxas aplicáveis poderão variar, de acordo com a política de crédito do PicPay Bank ou do Original, a partir de 1,24% ao mês. Esta oferta é válida para usuários que anteciparem uma parcela do saque-aniversário FGTS, via PicPay Bank ou Banco Original. para as contratações aprovadas pelo PicPay Bank e para as contratações aprovadas pelo Digio em dias úteis até às 17h. As antecipações que necessitem de uma análise detalhada de crédito e/ou complementação de cadastro ou que sejam aprovadas pelo Digio aos sábados, domingos e feriados ou em dias úteis após às 17 horas serão creditadas em até 1 dia útil. Consulte demais condições no momento da contratação."}

# Exemplo de email FRAUDE
dados3 = {"descricao":"Netflix Atualize seus dados de pagamento! Olá  Ocorreu um problema com seu meio de pagamento atual. Tentaremos efetuar uma cobrança novamente mas recomendamos que você atualize essas informações. ATUALIZAR PAGAMENTO Lembrando que sua conta estará suspensa caso o débito não seja realizado. Informações da sua conta: Email  Provedor do serviço Netflix Entretenimento Brasil, Ltda. Plano Básico Preço do plano R$21,90/mês Estamos sempre prontos para ajudar. Acesse o Centro de ajuda para saber mais ou fale com a gente. Seus amigos da Netflix VEJA TODAS AS SÉRIES E FILMES> Ao assinar a Netflix, você concorda com nossos Termos de uso e com a Declaração de privacidade Dúvidas? Ligue 0800-887-0201 Estamos enviando este email porque você é assinante Netflix. Para alterar suas preferências de email a qualquer momento, acesse a página Configurações de comunicação da sua conta. Não responda a este email, pois a caixa deste endereço de email não é monitorada. Para obter ajuda ou entrar em contato, acesse o Centro de ajuda em help.netflix.com. Esta mensagem foi enviada para [ erik1605@hotmail.com  ] pela Netflix. SRC: 12500_pt-BR_BR A utilização do serviços e do site da Netflix está sujeita aos Termos de privacidade. Netflix Entretenimento Brasil, Ltda."}

dados4 = {"descricao" : "Sua conta de e-mail foi infectada com um vírus de Trojan: para excluir o vírus e também proteger sua conta, Clique Aqui para atualizar o Administrador do Sistema."}


In [195]:
modelo = joblib.load('modelo.joblib')

In [196]:
# Pré-processa a mensagem
text_processado = process_text_conteudo(dados1["descricao"])
dados_transformados = vectorizer.transform([text_processado])

# Prediz se é fraude
predicao = modelo.predict(dados_transformados)
print("FRAUDE" if predicao[0] == 1 else "OK")

OK


In [197]:
# Pré-processa a mensagem
text_processado = process_text_conteudo(dados2["descricao"])
dados_transformados = vectorizer.transform([text_processado])

# Prediz se é fraude
predicao = modelo.predict(dados_transformados)
print("FRAUDE" if predicao[0] == 1 else "OK")

OK


In [198]:
# Pré-processa a mensagem
text_processado = process_text_conteudo(dados3["descricao"])
dados_transformados = vectorizer.transform([text_processado])

# Prediz se é fraude
predicao = modelo.predict(dados_transformados)
print("FRAUDE" if predicao[0] == 1 else "OK")

FRAUDE


In [199]:
# Pré-processa a mensagem
text_processado = process_text_conteudo(dados4["descricao"])
dados_transformados = vectorizer.transform([text_processado])

# Prediz se é fraude
predicao = modelo.predict(dados_transformados)
print("FRAUDE" if predicao[0] == 1 else "OK")

FRAUDE


In [200]:
# Pré-processa a mensagem
text_processado = process_text_conteudo(dados5["descricao"])
dados_transformados = vectorizer.transform([text_processado])

# Prediz se é fraude
predicao = modelo.predict(dados_transformados)
print("FRAUDE" if predicao[0] == 1 else "OK")

FRAUDE
